# 0 Information

## 0.1 Prerequisites

### 0.1.1 Git
* Should be installed
* Should be available via path, so that it is possible to call `git clone ...` directly from the console

### 0.1.2 Python
It is recommended to use version 3.8+. The oldest version on which
this notebook and all codebase scripts have been tested is 3.7.

### 0.1.3 tar
This tool is required if you want to download the dataset and unzip it

## 0.2 Recommendations

### 0.2.1 Configuration
The current setup of this file allow you to completely run all the modules of the project without any problems.
If you wish to simply execute all the functions, and you don't require special configuration or output directories
then please stick to the configuration that is provided here by default.

### 0.2.2 Google colab
A lot of scripts from here are very slow and it is advisable to run them from Google Colab. Also the whole testing was done on colab so it is a guarantee that everything works on Colab as intended. In Colab you don't have to care about virtual environments and packages in python.

### 0.2.3 Virtual environment
If you still decide to run the notebook locally, it is recommended to run this notebook in a separate `conda` python virtual environment.
Even though this is not a hard requirement to run this notebook it is very recommended using a virtual environment.

## 0.3 App
This notebook does not contain the startup of the frontend for the animal detector app. For this, follow the instructions from the readme.md file of the github repo for this project.

# 1 Setup

## 1.1 Constants

In [1]:
MODE_VERBOSE = 'verbose'
MODE_SILENT = 'silent'

DEFAULT_TRAINED_MODEL = 'my_model'

DEFAULT_CONFIG_FILE = 'config_default.json'

GITHUB_REPO_URL = 'https://github.com/Maks0bs/study-project-cv-real-vs-virtual.git'

## 1.2 Adjustable config constants
Change these if you want to experiment with the functionality of this notebook

### `SCRIPTS_EXEC_MODE`
It is possible to execute all scripts in `verbose` mode to display
all logs that are displayed by the scripts themselves
and also by all the subprocesses that they start. However, if you use `silent` mode
then no logs will be displayed. Only important messages like errors and
important execution checkpoints will be logged in silent mode.

In [2]:
SCRIPTS_EXEC_MODE = MODE_VERBOSE # MODE_SILENT

### `EVAL_SPLIT_RATIO`
Specify a float value from `0.0` to `1.0` which defines how big the part of images should
be used for evaluation. Value of `0.2` means that 20% of all the images in the provided dataset
should be used for evaluation. The rest is used for training

In [3]:
EVAL_SPLIT_RATIO = 0.2

### `NUM_TRAIN_STEPS`
The number of training steps during the model training process. See TFOD API documentation for details.

In [4]:
NUM_TRAIN_STEPS = 20000

### `CHECKPOINT_EVERY_N`
How often should Tensorflow make checkpoints of training status. See TFOD API documentation for details.

In [5]:
CHECKPOINT_EVERY_N = 1000

### `EXISTING_MODEL`

If you do not need to train the model, please specify the name of the trained model here and put its weights in the model

In [6]:
EXISING_MODEL = None # set to DEFAULT_TRAINED_MODEL if my_model is present and no training is needed

### `DOWNLOAD_DATASET`
Set to `True` if the dataset should be downloaded from the servers (https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz and https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz)

In [7]:
DOWNLOAD_DATASET = True

# 2 Workspace preparation
All of these cells should be run only for one time when we "start from scratch" with this notebook
Install the project package to the python (virtual) environment

In [13]:
!git clone -b animal-detection {GITHUB_REPO_URL} ./animal_detection

import shutil
shutil.copy('./animal_detection/setup.py', './setup.py')

Cloning into './animal_detection'...


'./setup.py'

In [8]:
import pip
pip.main(['install', '-e', '.'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Obtaining file:///D:/Programming/Projects/python/ksunia/animal-breed-detection/animal_detection

Preparing metadata (setup.py): started

Preparing metadata (setup.py): finished with status 'done'

Installing collected packages: animal-detection

...

Uninstalling animal-detection-1.0:

Successfully uninstalled animal-detection-1.0

Running setup.py develop for animal-detection

Successfully installed animal-detection-1.0

0

In [15]:
%cd ./animal_detection/

d:\Programming\Projects\python\ksunia\animal-breed-detection\animal_detection


## 2.1 Download archive with data

In [21]:
if DOWNLOAD_DATASET:
    import urllib, os
    urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz", "annotations.tar.gz")
    print('downloaded annotations')
    urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz", "images.tar.gz")
    print('downloaded images')
    
    os.makedirs('./data/datasets/raw/my_dataset/annotations', exist_ok=True)
    os.makedirs('./data/datasets/raw/my_dataset/images', exist_ok=True)
    !tar -zxf ./annotations.tar.gz --directory ./data/datasets/raw/my_dataset/annotations
    !tar -zxf ./images.tar.gz --directory ./data/datasets/raw/my_dataset/images
    

# 3 TFOD API and pretrained model
Here we install Tensorflow Object Detection API and
load the pretrained model that is specified in the config.
Generally, the TFOD API should only be installed once.
However, several pretrained models can be loaded (when the config is changed)
and in this case it is required to run these cells once again
for the new pretrained model to be used during training.

In [31]:
import os
python_file = os.path.join('setup', 'workspace.py')
! python {python_file} -m {SCRIPTS_EXEC_MODE} -v "both"

^C


c:\Users\makst\anaconda3\envs\animal_detection_env\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
c:\Users\makst\anaconda3\envs\animal_detection_env\lib\site-packages\setuptools\_distutils\cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
       

Verifying if TFOD API is installed (correctly)...
TFOD API is NOT installed (correctly)
object_detection folder exists, no need to clone again
Most likely Tensorflow Object detection folders are already structured.
If not, please check the folder where TFOD API should be installed for errors
        1 file(s) copied.
running build
running build_py
copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detect

In [28]:
# the code above installs some packages with incorrect versions and
# we need to reinstall these packages with correct versions
# for correct functionality of other parts of this notebook
! python -m pip install matplotlib scikit-learn pillow pytz gin-config tensorflow[and-cuda]==2.13.0 opencv-python

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
    --------------------------------------- 0.2/9.3 MB 4.6 MB/s eta 0:00:02
   - -------------------------------------- 0.3/9.3 MB 4.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/9.3 MB 4.0 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/9.3 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.3 MB 4.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.3 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/9.3 MB 4.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.3 MB 4.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.3 MB 4.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.3 MB 4.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/9.3 MB 4.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.5/9.3 MB 4.5 MB/s eta 0:00:02
   ----------- ----

## 4 Dataset preparation
Here we create the processed dataset with splitting into train and test sets.
Please put your raw data in ./data/datasets/raw/my_dataset if not done in the previous step

In [29]:
import os
python_file = os.path.join('data', 'preparation.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE} -sr {EVAL_SPLIT_RATIO}

Reading raw dataset
Successfully read raw dataset
Writing processed dataset
Successfully wrote processed dataset
generate tf records from d:\programming\projects\python\ksunia\animal-breed-detection\animal_detection/data/datasets/processed\my_dataset\train
records written successfully
generate tf records from d:\programming\projects\python\ksunia\animal-breed-detection\animal_detection/data/datasets/processed\my_dataset\eval
records written successfully


## 5 Train the model
Train the model on the created TF records

In [30]:
import os
python_file = os.path.join('models', 'train.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE} --num_train_steps {NUM_TRAIN_STEPS} --checkpoint_every_n {CHECKPOINT_EVERY_N}

loading pipeline config for training from the pretrained model...


2023-12-09 22:11:07.121001: E tensorflow/tsl/platform/windows/windows_file_system.cc:363] ERROR: GetSymbolicLinkTarget cannot open file for \\?\d:\programming\projects\python\ksunia\animal-breed-detection\animal_detection\models\pretrained\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\pipeline.config GetLastError: 3

Traceback (most recent call last):
  File "d:\Programming\Projects\python\ksunia\animal-breed-detection\animal_detection\models\train.py", line 232, in <module>
    execute(**kwargs)
  File "d:\Programming\Projects\python\ksunia\animal-breed-detection\animal_detection\models\train.py", line 218, in execute
    config_loader = write_training_config(mode)
  File "d:\Programming\Projects\python\ksunia\animal-breed-detection\animal_detection\models\train.py", line 194, in write_training_config
    config_loader.load_config_from_pretrained_model()
  File "d:\Programming\Projects\python\ksunia\animal-breed-detection\animal_detection\models\train.py", line 106, in load_config_fro

## 6 Evaluate the model

In [7]:
# TODO replace this with custom python script for eval

command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format('/content/animal_detection/tfod_api/object_detection/model_main_tf2.py', '/content/animal_detection/models/trained/my_model' ,'/content/animal_detection/models/trained/my_model/pipeline.config', '/content/animal_detection/models/trained/my_model')
command

'python /content/studienprojekt_cv_rvv/tfod_api/object_detection/model_main_tf2.py --model_dir=/content/studienprojekt_cv_rvv/models/trained/my_model --pipeline_config_path=/content/studienprojekt_cv_rvv/models/trained/my_model/pipeline.config --checkpoint_dir=/content/studienprojekt_cv_rvv/models/trained/my_model'

In [ ]:
!{command}

## 7 Image Object Detection

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# TODO replace this with custom python script for detection

PIPELINE_CONFIG_FILE = ''
LABELMAP_FILE = ''
NEW_CKPT_PATH = ''

configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_FILE)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(NEW_CKPT_PATH, 'ckpt-XXX')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP_FILE)
IMAGE_PATH = ''

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=.25,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.savefig('test.png')
plt.show()